# Feature importance

In [16]:
# essentials
import os
import pathlib

import pandas as pd
import numpy as np
from tqdm import tqdm

# visualisation
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn imports
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MaxAbsScaler, PowerTransformer, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2, f_classif, SequentialFeatureSelector, RFECV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.base import clone as clone_model

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, log_loss

# others
import xgboost as xgb 

RANDOM_SEED = 64

In [17]:
IN_KAGGLE = False
kaggle_folder = "/kaggle/input/playground-series-s3e26"
local_folder = "./data"
train_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "/train.csv", index_col="id")
test_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder  + "/test.csv", index_col="id")

target_column = "Status"

categorical_features = ["Drug", "Sex", "Ascites", "Hepatomegaly", "Spiders", "Edema", "Stage"]
numerical_features = ["N_Days", "Age", "Bilirubin", "Cholesterol", "Albumin", "Copper", "Alk_Phos", "SGOT", "Tryglicerides", "Platelets", "Prothrombin"]

# change categorical features to category type
for col in categorical_features:
    train_df[col] = train_df[col].astype("category")
    test_df[col] = test_df[col].astype("category")

# change "Stage" to string
train_df["Stage"] = train_df["Stage"].apply(lambda x: str(x))

X = train_df.drop(columns=target_column)
y = train_df[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=RANDOM_SEED, stratify=y, shuffle=True)

print(f"Number of training examples: {len(X_train)}")
print(f"Number of validation examples: {len(X_val)}")

print("Number of examples per class in training set")
print(y_train.value_counts())

print("Number of examples per class in validation set")
print(y_val.value_counts())

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

Number of training examples: 7114
Number of validation examples: 791
Number of examples per class in training set
Status
C     4468
D     2398
CL     248
Name: count, dtype: int64
Number of examples per class in validation set
Status
C     497
D     267
CL     27
Name: count, dtype: int64


In [18]:
log_dist = ['Bilirubin',   'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides',  'Prothrombin']
normal_dist = ['N_Days', 'Age', 'Cholesterol', 'Albumin', 'Platelets',]

numeric_transformer = Pipeline(
    [
        ("power_transformer", PowerTransformer()),
        ("scaler", MaxAbsScaler()),
    ]
)

categorical_transformer = Pipeline(
    [
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

model = xgb.XGBClassifier(objective="multi:softprob", random_state=RANDOM_SEED, n_jobs=-1)
clf = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", CalibratedClassifierCV(model, cv=skf)),
    ]
)

clf.fit(X_train, y_train)
y_pred_proba = clf.predict_proba(X_val)
y_pred = clf.predict(X_val)

log_loss_score = log_loss(y_val, y_pred_proba)

print(f"Log loss score: {log_loss_score:.4f}")

Log loss score: 0.4639


## Decision tree feature importances

In [19]:
# feature importance from CalibratedClassifierCV with cv=3
# create a dataframe with feature importance where rows are features and columns are folds

def generate_feature_importances_df(clf: Pipeline):
    importances_fold_1 = clf.named_steps["classifier"].calibrated_classifiers_[0].estimator.feature_importances_
    importances_fold_2 = clf.named_steps["classifier"].calibrated_classifiers_[1].estimator.feature_importances_
    importances_fold_3 = clf.named_steps["classifier"].calibrated_classifiers_[2].estimator.feature_importances_

    feature_names = clf.named_steps["preprocessor"].transformers_[1][1].named_steps["onehot"].get_feature_names_out()
    feature_names = np.concatenate([numerical_features, feature_names])
    feature_importance_df = pd.DataFrame({"feature": feature_names, "importance fold 1": importances_fold_1, "importance fold 2": importances_fold_2, "importance fold 3": importances_fold_3})
    feature_importance_df["mean importance"] = feature_importance_df[["importance fold 1", "importance fold 2", "importance fold 3"]].mean(axis=1)
    return feature_importance_df.sort_values("mean importance", ascending=False)

generate_feature_importances_df(clf)

,feature,importance fold 1,importance fold 2,importance fold 3,mean importance
2,Bilirubin,0.155204,0.170424,0.162455,0.162694
23,Edema_Y,0.111438,0.075448,0.123859,0.103582
27,Stage_4.0,0.081069,0.082218,0.099467,0.087585
21,Edema_N,0.081943,0.082888,0.061794,0.075542
17,Hepatomegaly_N,0.059864,0.084857,0.064474,0.069732
0,N_Days,0.049645,0.054895,0.043195,0.049245
10,Prothrombin,0.039072,0.039392,0.036604,0.038356
19,Spiders_N,0.034145,0.040622,0.038132,0.037633
15,Ascites_N,0.024687,0.013474,0.065813,0.034658
13,Sex_F,0.042474,0.028295,0.025500,0.032090


## Importances after feature engineering

In [20]:
def feature_engineering(df):
    #train_df['Status'] = train_df['Status'].map({"D": 0,"C": 1,"CL": 2})
    df['date_of_diagnosis'] = df['Age'] - df['N_Days']
    df['diseases'] = df['Ascites'] + df['Hepatomegaly'] + df['Spiders'] + df['Edema']

    # change categorical features to category type
    for col in categorical_features:
        df[col] = df[col].astype("category")

    # change "Stage" to string
    #df["Stage"] = df["Stage"].apply(lambda x: str(x))
    return df

train_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "/train.csv", index_col="id")

train_df = feature_engineering(train_df)

categorical_features = ["Drug", "Sex", "Ascites", "Hepatomegaly", "Spiders", "Edema",  "diseases"]
numerical_features = ["N_Days", "Age", "Bilirubin", "Cholesterol", "Albumin", "Copper", "Alk_Phos", "SGOT", "Tryglicerides", "Platelets", "Prothrombin", "date_of_diagnosis", "Stage"]

X = train_df.drop(columns=target_column)
y = train_df[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=RANDOM_SEED, stratify=y, shuffle=True)

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

numeric_transformer = Pipeline(
    [
        ("power_transformer", PowerTransformer()),
        ("scaler", MaxAbsScaler()),
    ]
)

categorical_transformer = Pipeline(
    [
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)

model = xgb.XGBClassifier(objective="multi:softprob", random_state=RANDOM_SEED, n_jobs=-1)
clf = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", CalibratedClassifierCV(model, cv=skf)),
    ]
)

clf.fit(X_train, y_train)
y_pred_proba = clf.predict_proba(X_val)
y_pred = clf.predict(X_val)

log_loss_score = log_loss(y_val, y_pred_proba)

print(f"Log loss score: {log_loss_score:.4f}")

Log loss score: 0.4653


In [21]:
generate_feature_importances_df(clf)

,feature,importance fold 1,importance fold 2,importance fold 3,mean importance
2,Bilirubin,0.119045,0.124094,0.131509,0.124883
29,diseases_NNNN,0.066150,0.075829,0.118018,0.086665
22,Edema_N,0.054074,0.078686,0.036004,0.056255
28,Stage_4.0,0.053111,0.059866,0.049330,0.054103
24,Edema_Y,0.050653,0.042221,0.063865,0.052247
16,Ascites_N,0.039459,0.011145,0.069504,0.040036
18,Hepatomegaly_N,0.028925,0.063187,0.024963,0.039025
0,N_Days,0.036706,0.037892,0.035324,0.036641
30,diseases_NNNS,0.023489,0.041341,0.033819,0.032883
36,diseases_NYNS,0.056692,0.016301,0.022778,0.031924


# Encoding features with 2 values as 0 and 1

In [22]:
# from train_df create a Dataframe showing the number of unique values in each feature in form of "column_name": number_of_unique_values
def get_unique_values_df(df: pd.DataFrame):
    unique_values = {}
    for col in df.columns:
        unique_values[col] = df[col].nunique()
    return pd.DataFrame({"feature": list(unique_values.keys()), "unique values": list(unique_values.values())})

get_unique_values_df(train_df)

,feature,unique values
0,N_Days,461
1,Drug,2
2,Age,391
3,Sex,2
4,Ascites,2
5,Hepatomegaly,2
6,Spiders,2
7,Edema,3
8,Bilirubin,111
9,Cholesterol,226


In [23]:
uniqueness = get_unique_values_df(train_df)
two_value_features = uniqueness[ uniqueness["unique values"] == 2 ]['feature'].values.tolist()
two_value_features

['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders']

## Feature selection

In [24]:
def feature_engineering(df):
    #train_df['Status'] = train_df['Status'].map({"D": 0,"C": 1,"CL": 2})
    df['date_of_diagnosis'] = df['Age'] - df['N_Days']
    
    df['no_diseases'] = (df['Ascites'] + df['Hepatomegaly'] + df['Spiders'] + df['Edema']) == 0
    df['diseases'] = df['Ascites'] + df['Hepatomegaly'] + df['Spiders'] + df['Edema']
    df['Drug'] = df['Drug'].map({"D-penicillamine": 1,"placebo": 0})

    # change categorical features to category type
    for col in categorical_features:
        df[col] = df[col].astype("category")

    # change "Stage" to string
    df["Stage"] = df["Stage"].apply(lambda x: str(x))
    return df

train_df = pd.read_csv(kaggle_folder if IN_KAGGLE else local_folder + "/train.csv", index_col="id")

run_feature_engineering = True

categorical_features = ["Drug", "Sex", "Ascites", "Hepatomegaly", "Spiders", "Edema", "Stage"]
numerical_features = ["N_Days", "Age", "Bilirubin", "Cholesterol", "Albumin", "Copper", "Alk_Phos", "SGOT", "Tryglicerides", "Platelets", "Prothrombin"]

if run_feature_engineering:
    train_df = feature_engineering(train_df)

if run_feature_engineering:
    categorical_features += ["no_diseases", "diseases"]
    numerical_features += ["date_of_diagnosis"]

X = train_df.drop(columns=target_column)
y = train_df[target_column]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=RANDOM_SEED, stratify=y, shuffle=True)

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)

numeric_transformer = Pipeline(
    [
        ("power_transformer", PowerTransformer()),
        ("scaler", MaxAbsScaler()),
    ]
)

categorical_transformer = Pipeline(
    [
        ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary")),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_SEED)


model = xgb.XGBClassifier(
    objective="multi:softprob", random_state=RANDOM_SEED, n_jobs=-1,
    subsample=0.8,
    min_child_weight=7,
    max_depth=7,
    reg_lambda=0.9,
    gamma=0.9,
    eta=0.08,
    colsample_bytree=0.5,
    reg_alpha=0.5
    )
clf = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", CalibratedClassifierCV(model, cv=skf)),
    ]
)

X_transformed = preprocessor.fit_transform(X_train)
X_transformed = pd.DataFrame(X_transformed, columns=numerical_features + preprocessor.transformers_[1][1].named_steps["onehot"].get_feature_names_out().tolist())


rfecv = RFECV(
    estimator=model,
    step=1,
    cv=skf,
    scoring="recall_micro",
    min_features_to_select=5,
    n_jobs=-1,
)

rfecv.fit(X_transformed, y_train)
optimal_features = X_transformed.columns[rfecv.support_]

print(f"Optimal number of features: {len(optimal_features)}")
print(f"Optimal features: {optimal_features}")


clf.fit(X_train, y_train)
y_pred_proba = clf.predict_proba(X_val)
y_pred = clf.predict(X_val)

log_loss_score = log_loss(y_val, y_pred_proba)
print(f"Log loss score with these features: {log_loss_score:.4f}")

Optimal number of features: 20
Optimal features: Index(['N_Days', 'Age', 'Bilirubin', 'Cholesterol', 'Copper', 'Alk_Phos',
       'SGOT', 'Platelets', 'Prothrombin', 'date_of_diagnosis', 'Sex_M',
       'Ascites_Y', 'Hepatomegaly_Y', 'Spiders_Y', 'Edema_N', 'Edema_Y',
       'Stage_1.0', 'Stage_4.0', 'diseases_NNNN', 'diseases_NYNN'],
      dtype='object')
Log loss score with these features: 0.4379


/home/wiktor/.pyenv/versions/3.10.11/envs/kaggling/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/wiktor/.pyenv/versions/3.10.11/envs/kaggling/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [8] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


## Permutation importance

In [25]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    rf, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2
)

sorted_importances_idx = result.importances_mean.argsort()
importances = pd.DataFrame(
    result.importances[sorted_importances_idx].T,
    columns=X.columns[sorted_importances_idx],
)
ax = importances.plot.box(vert=False, whis=10)
ax.set_title("Permutation Importances (test set)")
ax.axvline(x=0, color="k", linestyle="--")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()

NameError: name 'rf' is not defined

## Numerical features before & after transformation

In [ ]:
# display distribution of numerical features before transformations - 3 x 4

fig, ax = plt.subplots(3, 4, figsize=(20, 15))

ax = ax.flatten()

for i, col in enumerate(numerical_features):
    sns.histplot(data=train_df, x=col, ax=ax[i])
    ax[i].set_title(f"{col} distribution")

plt.tight_layout()
plt.show()

In [ ]:
numeric_transformer = Pipeline(
    [
        ("power_transformer", PowerTransformer()),
        ("scaler", MaxAbsScaler()),
    ]
)

post_transform_df = pd.DataFrame(numeric_transformer.fit_transform(train_df[numerical_features]), columns=numerical_features, index=train_df[numerical_features].index)

# display distribution of numerical features before transformations - 3 x 4

fig, ax = plt.subplots(3, 4, figsize=(20, 15))

ax = ax.flatten()

for i, col in enumerate(numerical_features):
    sns.histplot(data=post_transform_df, x=col, ax=ax[i])
    ax[i].set_title(f"{col} distribution")

plt.tight_layout()
plt.show()